In [1]:
import sys
sys.path.append("..")
import torch
import timm
from torch import nn
from torch import optim
import torchvision.transforms as transforms
import time
import os
from Tensorized_components.patch_embedding  import Patch_Embedding     
from Tensorized_components.w_msa  import WindowMSA     
from Tensorized_components.sh_wmsa  import ShiftedWindowMSA     
from Tensorized_components.patch_merging  import TensorizedPatchMerging  
from Tensorized_Layers.TCL_CHANGED import TCL_CHANGED   
from Tensorized_Layers.TRL import TRL   
from Utils.Accuracy_measures import topk_accuracy
from Utils.TinyImageNet_loader import get_tinyimagenet_dataloaders
from Utils.Num_parameter import count_parameters

# Define image transforms for training and validation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])



# Setup the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(f'Device is set to : {device}')

# Configs

TEST_ID = 'Test_ID005'
batch_size = 5
n_epoch = 400
image_size = 224

# Set up the transforms and train/test loaders

tiny_transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize((image_size, image_size)), 
        transforms.RandomCrop(image_size, padding=5),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
tiny_transform_val = transforms.Compose([
        transforms.Resize((image_size, image_size)), 
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
tiny_transform_test = transforms.Compose([
        transforms.Resize((image_size, image_size)), 
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])


train_loader, test_loader , _ = get_tinyimagenet_dataloaders(
                                                    data_dir = '../datasets',
                                                    transform_train=tiny_transform_train,
                                                    transform_val=tiny_transform_val,
                                                    transform_test=tiny_transform_test,
                                                    batch_size=batch_size,
                                                    image_size=image_size)


# Create the Tiny Swin Transformer model with timm.
# "swin_tiny_patch4_window7_224" is the tiny variant.
# We set num_classes=200 to match Tiny ImageNet.
model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=False, num_classes=200)

# Calculate and print the total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters:", total_params)

# # Set up device and move the model to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Define loss function and optimizer
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# # A simple training loop for one epoch
# model.train()
# for images, labels in train_loader:
#     images, labels = images.to(device), labels.to(device)
    
#     optimizer.zero_grad()
#     outputs = model(images)
#     loss = criterion(outputs, labels)
#     loss.backward()
#     optimizer.step()
    
#     print("Loss:", loss.item())


c:\Users\m.badzohreh\AppData\Local\anaconda3\envs\data_science\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device is set to : cpu
Total number of parameters: 27673154
